In [ ]:
#Determinismo



import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
import numpy as np
import re
import gc
from collections import Counter

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    f1_score,
    roc_auc_score,
    average_precision_score,
    classification_report
)

# Transformers
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (
    RobertaTokenizer,
    AutoTokenizer,
    RobertaTokenizerFast,
    RobertaModel,
    RobertaForSequenceClassification,
    BertForSequenceClassification,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)

# Datasets
from datasets import Dataset, load_dataset, DatasetDict

# Optuna
import optuna
import wandb

# Autologin a W&B
wandb.login(key="59f9600e6173b66ce484fd9c8f0debf8b76f69e7")


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ddorado/.netrc
wandb: Currently logged in as: danieldorado1996 (danieldorado1996-university-of-huelva) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

##Funciones

In [ ]:

#Evaluar Metricas
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)
    probs = predictions[:, 1]  # Asumiendo binario, clase positiva = columna 1

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_mayoritaria = f1_score(labels, preds, pos_label=1)
    f1_minoritaria = f1_score(labels, preds, pos_label=0)

    # Aquí el cambio: AUC y average precision con probs, no con preds
    try:
        auc = roc_auc_score(labels, probs)
        prec_rec = average_precision_score(labels, probs)
    except ValueError:
        # Esto puede pasar si labels no contienen ambas clases
        auc = float('nan')
        prec_rec = float('nan')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1_macro,
        'AUC': auc,
        'PREC_REC': prec_rec,
        'f1_minoritaria': f1_minoritaria,
        'f1_mayoritaria': f1_mayoritaria,
    }
file_path = 'trans_def.csv'

df = pd.read_csv(file_path)

#limpiar Vacios
df['about'] = df['about'].fillna('Vacia')
df['goal'] = df['goal'].fillna('Vacia')
df['description'] = df['description'].fillna('Vacia')
df['motivation'] = df['motivation'].fillna('Vacia')

#Crear dsataset a poartir de tabla
def generarTextTemplate(row, opc=0):
    text = ""


    if opc == 0 or opc == 1:
        if row['about'] is not None and pd.notna(row['about']):
            text += row['about'] + "."
            print(1)


    if opc == 0 or opc == 2:
        if row['description'] is not None and pd.notna(row['description']):
            text += row['description'] + "."
            print(2)

    if opc == 0 or opc == 3:
        if row['motivation'] is not None and pd.notna(row['motivation']):
            text += row['motivation'] + "."
            print(3)

    if opc == 0 or opc == 4:
        if row['goal'] is not None and pd.notna(row['goal']):
            text += str(row['goal']) + "."
            print(4)

    return pd.Series({'text': text, 'label': 1 if row['status'] == 'SI' else 0})


#adapta lo necesario para generar  los conjuntos traint test y valid
def preparar_datasets(df, dft,w ,st=3, model_name='PlanTL-GOB-ES/longformer-base-4096-bne-es'):

    df2 = pd.DataFrame([generarTextTemplate(df.iloc[i],w) for i in range(len(df))])
    dft2 = pd.DataFrame([generarTextTemplate(dft.iloc[i],w) for i in range(len(dft))])

# Separar el dataframe por clase
    df_clase_0 = df2[df2['label'] == 0]  # Clase minoritaria
    df_clase_1 = df2[df2['label'] == 1]  # Clase mayoritaria

# Número de ejemplos de la clase minoritaria (0)
    n_0 = len(df_clase_0)



    n_1_deseado = n_0 * st  # Número deseado de ejemplos de clase 1

# Hacer undersampling de la clase mayoritaria
    df_clase_1_downsampled = df_clase_1.sample(n=n_1_deseado, replace=False, random_state=42)

# Concatenar las dos clases después de hacer el undersampling
    df_balanced = pd.concat([df_clase_0, df_clase_1_downsampled])

# Mezclar el conjunto de datos balanceado
    df2_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
    dataset = Dataset.from_pandas(df2_balanced)
    datasett = Dataset.from_pandas(dft2)
# Verifica el límite del modelo cargado
    model_max_length = tokenizer.model_max_length

    def tokenize_function(example):
        return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=min(model_max_length, 512),
        return_attention_mask=True
    )





    # Aplicar tokenización
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_datasett = datasett.map(tokenize_function, batched=True)

    # Convertir a tensores
    tokenized_dataset = tokenized_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])
    tokenized_datasett = tokenized_datasett.with_format("torch", columns=["input_ids", "attention_mask", "label"])

    # Dividir datasets
    split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
    train_data = split_dataset['train']
    valid_data = split_dataset['test']
    test_data = tokenized_datasett

    return train_data, valid_data, test_data




#optuna
def objective(trial):
    global modelN
    global bauc
    global v
    global path

    # Limpiar la memoria de la GPU antes de cada nueva iteración
    torch.cuda.empty_cache()
    gc.collect()

    # Selección de parámetros para la optimización
    learning_rate = trial.suggest_categorical("learning_rate",  [5e-6,1e-5, 2e-5, 3e-5, 5e-5,1e-4])
    weight_decay = trial.suggest_categorical("weight_decay", [0.1, 0.01, 0.001])
    optimizer = trial.suggest_categorical("optimizer", [ "adamw_torch", "adafactor", "schedule_free_radam"])
    bs = trial.suggest_categorical("batch_size", [16,32])


    tokenizer = AutoTokenizer.from_pretrained(modelN, do_lower_case=True)

    model = AutoModelForSequenceClassification.from_pretrained(
        modelN, num_labels=2, force_download=True)


    training_args = TrainingArguments(
        save_strategy="no",
        logging_strategy="no",
        eval_strategy="epoch",

        learning_rate=learning_rate,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        num_train_epochs=10,
        weight_decay=weight_decay,

        metric_for_best_model='AUC',

        optim=optimizer,
        fp16=False,
        dataloader_num_workers=0,

    )

    # Inicializar el trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=valid_data,
        tokenizer=tokenizer,
         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] ,
        compute_metrics=compute_metrics  # Incluir cálculo de F1
    )

    # Entrenar el modelo
    trainer.train()


    with torch.no_grad():

        # Obtener las predicciones y calcular el F1 score
        predictions, labels, _ = trainer.predict(test_data)
        preds = predictions.argmax(axis=-1)

        probs = predictions[:, 1]

        df = pd.DataFrame({
        'label': labels,
        'predicted_label': preds,
        'probability_class_1': probs
      })

    # Guardar como CSV
        df.to_csv(f"{path}_preds", index=False)
        preds = predictions.argmax(axis=-1)
        probs = predictions[:, 1]

        accuracy = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
        f1_macro = f1_score(labels, preds, average='macro')
        f1_mayoritaria = f1_score(labels, preds, pos_label=1)
        f1_minoritaria = f1_score(labels, preds, pos_label=0)



        auc = roc_auc_score(labels, probs)
        prec_rec = average_precision_score(labels, probs)


    trial.set_user_attr("accuracy", accuracy)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1-macro", f1_macro)
    trial.set_user_attr("f1_minoritaria", f1_minoritaria)
    trial.set_user_attr("f1_mayoritaria", f1_mayoritaria)
    trial.set_user_attr("AUC", auc)
    trial.set_user_attr("PREC_REC", prec_rec)
    if auc>bauc:
      bauc=auc

      print(auc," se  guardo   \n")

      model_path = path


      trainer.save_model(model_path)

      trainer.tokenizer.save_pretrained(model_path)


    else:
            print(auc," no se  guardo   \n")
    # Liberar memoria de los objetos trainer y modelo
    del model
    del trainer


    return auc





"\n    df_clase_0 = dft2[dft2['label'] == 0]  # Clase minoritaria\n    df_clase_1 = dft2[dft2['label'] == 1]  # Clase mayoritaria\n\n# Número de ejemplos de la clase minoritaria (0)\n    n_0 = len(df_clase_0)\n     \n\n# Queremos 3 veces más ejemplos de la clase 1 que de la clase 0\n    n_1_deseado = n_0 * 3  # Número deseado de ejemplos de clase 1\n\n# Hacer undersampling de la clase mayoritaria (1) para obtener la proporción 3:1\n    df_clase_1_downsampled = df_clase_1.sample(n=n_1_deseado, replace=False, random_state=42)\n\n# Concatenar las dos clases después de hacer el undersampling\n    df_balanced = pd.concat([df_clase_0, df_clase_1_downsampled])\n\n# Mezclar el conjunto de datos balanceado\n    dft2_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)\n    "

##Busqueda Proporcion

In [ ]:
#dividir train test
dftr, dft = train_test_split(
    df,
    test_size=0.2,
    stratify=df['status'],
    random_state=42
)




In [ ]:

for il in range(1,6):
    for v in range(1,5):
        ruta=None
        train_data, valid_data, test_data=preparar_datasets(dftr, dft,v,st=il, model_name= "'dccuchile/bert-base-spanish-wwm-uncased'")

        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"

        print(f"Dispositivo disponible: {torch.cuda.current_device()} ({torch.cuda.get_device_name()})")

        learning_rate = 1e-5
        weight_decay = 0.01
        optimizer = "adafactor"
        bs = 16


        tokenizer = AutoTokenizer.from_pretrained( "'dccuchile/bert-base-spanish-wwm-uncased'", do_lower_case=True)

        model = AutoModelForSequenceClassification.from_pretrained(
         "'dccuchile/bert-base-spanish-wwm-uncased'", num_labels=2,force_download=True)


        training_args = TrainingArguments(

        eval_strategy="epoch",

        learning_rate=learning_rate,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        num_train_epochs=10,
        weight_decay=weight_decay,
        save_strategy="no",
        logging_strategy="no",
        metric_for_best_model='AUC',

        optim=optimizer,
        fp16=False,
        dataloader_num_workers=0,

    )

    # Inicializar el trainer
        trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=valid_data,
        tokenizer=tokenizer,
         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] ,
        compute_metrics=compute_metrics  # Incluir cálculo de F1
    )

    # Entrenar el modelo
        trainer.train()


        with torch.no_grad():


            predictions, labels, _ = trainer.predict(test_data)
            preds = predictions.argmax(axis=-1)

        probs = predictions[:, 1]


        df = pd.DataFrame({
        'label': labels,
        'predicted_label': preds,
        'probability_class_1': probs
      })




        preds = predictions.argmax(axis=-1)
        probs = predictions[:, 1]

        accuracy = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
        f1_macro = f1_score(labels, preds, average='macro')
        f1_mayoritaria = f1_score(labels, preds, pos_label=1)
        f1_minoritaria = f1_score(labels, preds, pos_label=0)


        auc = roc_auc_score(labels, probs)
        prec_rec = average_precision_score(labels, probs)
        print(f"Campo {v} Tamano {il}    :  {auc}")





        model_path = f"./BtTam/Bt{il}-{v}"

        del model
        del trainer








Map:   0%|          | 0/592 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Dispositivo disponible: 0 (NVIDIA GeForce RTX 4090)


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc,Prec Rec,F1 Minoritaria,F1 Mayoritaria
1,No log,0.695144,0.483146,0.241573,0.500000,0.325758,0.575708,0.603218,0.651515,0.000000
2,No log,0.694927,0.483146,0.241573,0.500000,0.325758,0.581775,0.609535,0.651515,0.000000


/home/ddorado/anaconda3/envs/cudaenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ddorado/anaconda3/envs/cudaenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

##Busqueda Optuna

In [ ]:

dftr, dft = train_test_split(
    df,
    test_size=0.2,
    stratify=df['status'],
    random_state=42
)




In [ ]:
#Busqueda de Hiperparametros
modelN='dccuchile/bert-base-spanish-wwm-uncased'
for v in range(1,5):

    path = f'./Bt/Bt{v}.csv'

    train_data, valid_data, test_data = preparar_datasets(
        dftr, dft, v, 2,model_name=modelN
    )

    bauc = 0


    # Limpieza de GPU
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    print(f"Dispositivo disponible: {torch.cuda.current_device()} ({torch.cuda.get_device_name()})")

    # Crear el estudio de Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=15)

    df = study.trials_dataframe()


    csv_filename = f"./Bt/optuna_Bt{v}.csv"
    df.to_csv(csv_filename, index=False)


    best_trial = study.best_trial
    best_params = best_trial.params

    best_params["model_name"] = f'Bt{v}'
    best_params["objective_value"] = best_trial.value

    # CSV acumulativo
    csv_accum_path = "./best_trials_acumulado.csv"

    df_best = pd.DataFrame([best_params])

    if os.path.exists(csv_accum_path):
        df_existing = pd.read_csv(csv_accum_path)
        df_combined = pd.concat([df_existing, df_best], ignore_index=True, sort=True)
    else:
        df_combined = df_best

    df_combined.to_csv(csv_accum_path, index=False)

    print(f"Experimento {v} completado. Mejor trial guardado en {csv_accum_path}")

modelN="PlanTL-GOB-ES/roberta-base-bne"
for v in range(1,5):

    path = f'./Rb/Rb{v}.csv'

    train_data, valid_data, test_data = preparar_datasets(
        dftr, dft, v, 2,model_name=modelN
    )

    bauc = 0



    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    print(f"Dispositivo disponible: {torch.cuda.current_device()} ({torch.cuda.get_device_name()})")


    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=15)

    df = study.trials_dataframe()

    csv_filename = f"./Rb/optuna_Rb{v}.csv"
    df.to_csv(csv_filename, index=False)

    best_trial = study.best_trial
    best_params = best_trial.params


    best_params["model_name"] = f'Rb{v}'
    best_params["objective_value"] = best_trial.value

    # CSV acumulativo
    csv_accum_path = "./best_trials_acumulado.csv"

    df_best = pd.DataFrame([best_params])

    if os.path.exists(csv_accum_path):
        df_existing = pd.read_csv(csv_accum_path)
        df_combined = pd.concat([df_existing, df_best], ignore_index=True, sort=True)
    else:
        df_combined = df_best

    df_combined.to_csv(csv_accum_path, index=False)

    print(f"Experimento {v} completado. Mejor trial guardado en {csv_accum_path}")



##Entrenamiento Mejores Hiperparametros

In [ ]:
import os

df = pd.read_csv("best_trials_acumulado.csv")
df.columns = ['batch_size', 'learning_rate', 'experiment_id', 'score', 'optimizer', 'weight_decay']
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for idx, row in df.iterrows():
    batch_size = int(row['batch_size'])
    learning_rate = float(row['learning_rate'])
    optimizer_name = row['optimizer']
    weight_decay = float(row['weight_decay'])
    experiment_id = row['experiment_id']
    prefix = experiment_id[:2]
    num_id = int(experiment_id[2:])
    if experiment_id.startswith("Bt"):
        model_name = "dccuchile/bert-base-spanish-wwm-uncased"

    elif experiment_id.startswith("Rb"):
        model_name = "PlanTL-GOB-ES/roberta-base-bne"


    for i in range(0,3):

        train_data, valid_data, test_data=preparar_datasets(bootstraps[i], dft,w=num_id, model_name= model_name)

        print(f"Entrenando experimento {idx} con batch_size={batch_size}, lr={learning_rate}, optimizer={optimizer_name}, weight_decay={weight_decay}")
        tokenizer = AutoTokenizer.from_pretrained( model_name, do_lower_case=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Configuración de entrenamiento
        training_args = TrainingArguments(

        eval_strategy="epoch",

        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,            # Tamaño del lote de entrenamiento
        per_device_eval_batch_size=batch_size,             # Tamaño del lote de evaluación
        num_train_epochs=10,
        weight_decay=weight_decay,
        save_strategy="no",
        logging_strategy="no",
        metric_for_best_model='AUC',

        optim=optimizer_name,
        fp16=False,
        dataloader_num_workers=0,

    )

        trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=valid_data,
        tokenizer=tokenizer,
         callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] ,
        compute_metrics=compute_metrics  # Incluir cálculo de F1
    )

        trainer.train()
        with torch.no_grad():

        # Obtener las predicciones y calcular el F1 score
            predictions, labels, _ = trainer.predict(test_data)
            preds = predictions.argmax(axis=-1)

        probs = predictions[:, 1]  # Asumiendo binario, clase positiva = columna 1

     # Guardar etiquetas y predicciones en un DataFrame
        df = pd.DataFrame({
        'label': labels,
        'predicted_label': preds,
        'probability_class_1': probs
      })

    # Guardar como CSV
        df.to_csv(f"./{prefix}/{prefix}B{i}-M{num_id}_preds", index=False)


        preds = predictions.argmax(axis=-1)
        probs = predictions[:, 1]  # Asumiendo binario, clase positiva = columna 1

        accuracy = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
        f1_macro = f1_score(labels, preds, average='macro')
        f1_mayoritaria = f1_score(labels, preds, pos_label=1)
        f1_minoritaria = f1_score(labels, preds, pos_label=0)

    # Aquí el cambio: AUC y average precision con probs, no con preds

        auc = roc_auc_score(labels, probs)
        prec_rec = average_precision_score(labels, probs)
        print(f'{accuracy} {f1_macro} {auc}')





        model_path = f"./{prefix}/{prefix}B{i}-M{num_id}"  # Choose a path in your Google Drive


        trainer.save_model(model_path)

        trainer.tokenizer.save_pretrained(model_path)



    # Liberar memoria de los objetos trainer y modelo
        del model
        del trainer



##Evaluar Resultados

In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, average_precision_score
)
from scipy.special import expit

def evaluate_ensemble(files):
    logits_list = []
    labels = None

    for file in files:
        df = pd.read_csv(file )
        if labels is None:
            labels = df['label'].values
        else:
            assert np.array_equal(labels, df['label'].values), f"Labels mismatch in {file}"
        logits = df['probability_class_1'].values
        logits_list.append(logits)

    logits_array = np.array(logits_list)
    probs_array = expit(logits_array)
    ensemble_probs = np.mean(probs_array, axis=0)
    ensemble_preds = (ensemble_probs >= 0.5).astype(int)

    accuracy = accuracy_score(labels, ensemble_preds)
    precision = precision_score(labels, ensemble_preds, average='macro')
    recall = recall_score(labels, ensemble_preds, average='macro')
    f1_macro = f1_score(labels, ensemble_preds, average='macro')
    f1_pos = f1_score(labels, ensemble_preds, pos_label=1)
    f1_neg = f1_score(labels, ensemble_preds, pos_label=0)
    auc = roc_auc_score(labels, ensemble_probs)
    avg_prec = average_precision_score(labels, ensemble_probs)

    print("\n📊 Métricas del ensemble:")
    print(f"Accuracy:               {accuracy:.4f}")
    print(f"Precision (macro):      {precision:.4f}")
    print(f"Recall (macro):         {recall:.4f}")
    print(f"F1 Score (macro):       {f1_macro:.4f}")
    print(f"F1 clase positiva (1):  {f1_pos:.4f}")
    print(f"F1 clase negativa (0):  {f1_neg:.4f}")
    print(f"ROC AUC:                {auc:.4f}")
    print(f"Average Precision:      {avg_prec:.4f}")

    return ensemble_probs, labels

# Grupos de archivos (sin extensión)
file_groups = [
["./Rb/RbB0-M1_preds", "./Rb/RbB1-M1_preds", "./Rb/RbB2-M1_preds"],
["./Rb/RbB0-M2_preds", "./Rb/RbB1-M2_preds", "./Rb/RbB2-M2_preds"],
["./Rb/RbB0-M3_preds", "./Rb/RbB1-M3_preds", "./Rb/RbB2-M3_preds"],
["./Rb/RbB0-M4_preds", "./Rb/RbB1-M4_preds", "./Rb/RbB2-M4_preds"]
]

file_groups =   [ ["./Bt/BtB0-M1_preds", "./Bt/BtB1-M1_preds", "./Bt/BtB2-M1_preds"],
    ["./Bt/BtB0-M2_preds", "./Bt/BtB1-M2_preds", "./Bt/BtB2-M2_preds"],
    ["./Bt/BtB0-M3_preds", "./Bt/BtB1-M3_preds", "./Bt/BtB2-M3_preds"],
    ["./Bt/BtB0-M4_preds", "./Bt/BtB1-M4_preds", "./Bt/BtB2-M4_preds"]
]
all_ensemble_probs = []
all_labels = None

for idx, group in enumerate(file_groups, 1):
    print(f"\n==== Evaluando grupo {idx} ====")
    probs, labels = evaluate_ensemble(group)
    all_ensemble_probs.append(probs)
    if all_labels is None:
        all_labels = labels
    else:
        assert np.array_equal(all_labels, labels), "Labels mismatch between groups"

# Ensemble general con las probabilidades promedio de los 4 grupos
all_ensemble_probs_array = np.array(all_ensemble_probs)
final_ensemble_probs = np.mean(all_ensemble_probs_array, axis=0)
final_ensemble_preds = (final_ensemble_probs >= 0.5).astype(int)

accuracy = accuracy_score(all_labels, final_ensemble_preds)
precision = precision_score(all_labels, final_ensemble_preds, average='macro')
recall = recall_score(all_labels, final_ensemble_preds, average='macro')
f1_macro = f1_score(all_labels, final_ensemble_preds, average='macro')
f1_pos = f1_score(all_labels, final_ensemble_preds, pos_label=1)
f1_neg = f1_score(all_labels, final_ensemble_preds, pos_label=0)
auc = roc_auc_score(all_labels, final_ensemble_probs)
avg_prec = average_precision_score(all_labels, final_ensemble_probs)

print("\n==== Métricas del ENSEMBLE GENERAL (todos los boostraps) ====")
print(f"Accuracy:               {accuracy:.4f}")
print(f"Precision (macro):      {precision:.4f}")
print(f"Recall (macro):         {recall:.4f}")
print(f"F1 Score (macro):       {f1_macro:.4f}")
print(f"F1 clase positiva (1):  {f1_pos:.4f}")
print(f"F1 clase negativa (0):  {f1_neg:.4f}")
print(f"ROC AUC:                {auc:.4f}")
print(f"Average Precision:      {avg_prec:.4f}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    f1_score,
    roc_auc_score,
    average_precision_score
)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay

def evaluate_ensemble(files):
    probabilities = []
    predictions = []
    labels = None

    for file in files:
        df = pd.read_csv(file)
        probabilities.append(df['probability_class_1'].values)
        predictions.append(df['predicted_label'].values)
        if labels is None:
            labels = df['label'].values

    probs_array = np.array(probabilities)
    preds_array = np.array(predictions)
    ensemble_probs = np.mean(probs_array, axis=0)
    ensemble_preds = (ensemble_probs >= 0.5).astype(int)

    accuracy = accuracy_score(labels, ensemble_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, ensemble_preds, average='macro')
    auc = roc_auc_score(labels, ensemble_probs)
    ap = average_precision_score(labels, ensemble_probs)
    f1_minoritaria = f1_score(labels, ensemble_preds, pos_label=0)
    print("\n📊 Métricas del ensemble:")
    print(f"Accuracy:           {accuracy:.4f}")
    print(f"Precision:          {precision:.4f}")
    print(f"Recall:             {recall:.4f}")
    print(f"F1 Minoritaria:     {f1_minoritaria:.4f}")
    print(f"F1 Score (Macro):   {f1:.4f}")
    print(f"ROC AUC:            {auc:.4f}")
    print(f"Average Precision:  {ap:.4f}")

    return ensemble_preds, ensemble_probs, labels, auc

def majority_vote_with_auc(ensemble_preds_list, ensemble_probs_list, auc_scores, labels):
    preds_array = np.array(ensemble_preds_list)
    n_models, n_samples = preds_array.shape

    final_preds = []
    for i in range(n_samples):
        votes = preds_array[:, i]
        vote_count = np.bincount(votes)
        if len(vote_count) == 1 or vote_count[0] != vote_count[1]:
            final_preds.append(np.argmax(vote_count))
        else:
            best_model_idx = np.argmax(auc_scores)
            final_preds.append(ensemble_preds_list[best_model_idx][i])

    final_preds = np.array(final_preds)
    probs_array = np.array(ensemble_probs_list)
    final_probs = np.mean(probs_array, axis=0)

    accuracy = accuracy_score(labels, final_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, final_preds, average='macro')
    f1_mayoritaria = f1_score(labels, final_preds, pos_label=1)
    f1_minoritaria = f1_score(labels, final_preds, pos_label=0)
    auc = roc_auc_score(labels, final_probs)
    ap = average_precision_score(labels, final_probs)

    print("\n🔚 Ensemble final (votación + AUC en empates):")
    print(f"Accuracy:           {accuracy:.4f}")
    print(f"Precision:          {precision:.4f}")
    print(f"Recall:             {recall:.4f}")

    print(f"F1 Macro:           {f1:.4f}")
    print(f"F1 Mayoritaria:     {f1_mayoritaria:.4f}")
    print(f"F1 Minoritaria:     {f1_minoritaria:.4f}")
    print(f"ROC AUC:            {auc:.4f}")
    print(f"Average Precision:  {ap:.4f}")
     # Guardar resultados en CSV
    results_df = pd.DataFrame({
        'label': labels,
        'predicted_label': final_preds,
        'probability_class_1': final_probs
    })

    results_df.to_csv("ensemble_final_predictions.csv", index=False)
    print("\n📁 Resultados guardados en 'ensemble_final_predictions.csv'")
    fpr, tpr, _ = roc_curve(labels, final_probs)
    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC - Ensemble Final RoBERTa')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig("ensemble_final_roc_curve.png")
    plt.close()
    print("🖼️ Curva ROC guardada como 'ensemble_final_roc_curve.png'")

    # 📊 Guardar matriz de confusión
    cm = confusion_matrix(labels, final_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Matriz de Confusión - Ensemble Final RoBERTa')
    plt.savefig("ensemble_final_confusion_matrix.png")
    plt.close()
    print("🖼️ Matriz de confusión guardada como 'ensemble_final_confusion_matrix.png'")


# Archivos de predicciones para Bt
file_groups =   [ ["./Bt/BtB0-M1_preds", "./Bt/BtB1-M1_preds", "./Bt/BtB2-M1_preds"],
    ["./Bt/BtB0-M2_preds", "./Bt/BtB1-M2_preds", "./Bt/BtB2-M2_preds"],
    ["./Bt/BtB0-M3_preds", "./Bt/BtB1-M3_preds", "./Bt/BtB2-M3_preds"],
    ["./Bt/BtB0-M4_preds", "./Bt/BtB1-M4_preds", "./Bt/BtB2-M4_preds"]]


ensemble_preds_list = []
ensemble_probs_list = []
auc_scores = []
labels = None

# Evaluar cada ensemble individual (por modelo)
for group in file_groups:
    preds, probs, labels, auc = evaluate_ensemble(group)
    ensemble_preds_list.append(preds)
    ensemble_probs_list.append(probs)
    auc_scores.append(auc)

# Combinar todos los ensembles en uno final
majority_vote_with_auc(ensemble_preds_list, ensemble_probs_list, auc_scores, labels)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from scipy.special import expit
from pathlib import Path

# Lista de todos los modelos
file_paths = [
    "./Rb/Bt1-1_preds", "./Rb/Bt1-2_preds", "./Rb/Bt1-3_preds", "./Rb/Bt1-4_preds",
    "./Rb/Bt2-1_preds", "./Rb/Bt2-2_preds", "./Rb/Bt2-3_preds", "./Rb/Bt2-4_preds",
    "./Rb/Bt3-1_preds", "./Rb/Bt3-2_preds", "./Rb/Bt3-3_preds", "./Rb/Bt3-4_preds",
    "./Rb/Bt4-1_preds", "./Rb/Bt4-2_preds", "./Rb/Bt4-3_preds", "./Rb/Bt4-4_preds",
    "./Rb/Bt5-1_preds", "./Rb/Bt5-2_preds", "./Rb/Bt5-3_preds", "./Rb/Bt5-4_preds"
]

results = []

for path in file_paths:
    filepath = Path(path)
    df = pd.read_csv(filepath)
    labels = df['label'].values
    probs = expit(df['probability_class_1'].values)
    preds = (probs >= 0.5).astype(int)

    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average='macro')
    rec = recall_score(labels, preds, average='macro')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_pos = f1_score(labels, preds, pos_label=1)
    f1_neg = f1_score(labels, preds, pos_label=0)
    auc = roc_auc_score(labels, probs)
    avg_prec = average_precision_score(labels, probs)

    results.append({
        "Modelo": filepath.name,
        "Accuracy": acc,
        "Precision (macro)": prec,
        "Recall (macro)": rec,
        "F1 (macro)": f1_macro,
        "F1 clase 1": f1_pos,
        "F1 clase 0": f1_neg,
        "ROC AUC": auc,
        "Average Precision": avg_prec
    })

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by="Modelo").reset_index(drop=True)
results_df_sorted


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from scipy.special import expit
from pathlib import Path

# Lista de todos los modelos
file_paths = [
    "./Rb/Bt1-1_preds", "./Rb/Bt1-2_preds", "./Rb/Bt1-3_preds", "./Rb/Bt1-4_preds",
    "./Rb/Bt2-1_preds", "./Rb/Bt2-2_preds", "./Rb/Bt2-3_preds", "./Rb/Bt2-4_preds",
    "./Rb/Bt3-1_preds", "./Rb/Bt3-2_preds", "./Rb/Bt3-3_preds", "./Rb/Bt3-4_preds",
    "./Rb/Bt4-1_preds", "./Rb/Bt4-2_preds", "./Rb/Bt4-3_preds", "./Rb/Bt4-4_preds",
    "./Rb/Bt5-1_preds", "./Rb/Bt5-2_preds", "./Rb/Bt5-3_preds", "./Rb/Bt5-4_preds"
]

results = []

for path in file_paths:
    filepath = Path(path)
    df = pd.read_csv(filepath)
    labels = df['label'].values
    probs = expit(df['probability_class_1'].values)
    preds = (probs >= 0.5).astype(int)

    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average='macro')
    rec = recall_score(labels, preds, average='macro')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_pos = f1_score(labels, preds, pos_label=1)
    f1_neg = f1_score(labels, preds, pos_label=0)
    auc = roc_auc_score(labels, probs)
    avg_prec = average_precision_score(labels, probs)

    results.append({
        "Modelo": filepath.name,
        "Accuracy": acc,
        "Precision (macro)": prec,
        "Recall (macro)": rec,
        "F1 (macro)": f1_macro,
        "F1 clase 1": f1_pos,
        "F1 clase 0": f1_neg,
        "ROC AUC": auc,
        "Average Precision": avg_prec
    })

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by="Modelo").reset_index(drop=True)
results_df_sorted